# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

Number of establishments with a hygiene score of 20: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('679180266278f16cc5eba21f'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'

In [7]:
# Fetch all documents that match the query
results = establishments.find(query)

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df.shape[0]}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 41
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  679180266278f16cc5eba21f   110681                  0   
1  679180266278f16cc5eba5a0   612039                  0   
2  679180266278f16cc5eba8ac   730933                  0   
3  679180266278f16cc5ebaa99   172735                  0   
4  679180266278f16cc5ebaaa9   172953                  0   
5  679180266278f16cc5ebb447   512854                  0   
6  679180266278f16cc5ebb669  1537089                  0   
7  679180276278f16cc5ebcb93   155648                  0   
8  679180276278f16cc5ebcfd9  1012883                  0   
9  679180276278f16cc5ebd7ea   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza 

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": "London",
    "RatingValue": {"$gte": 4}
}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with a RatingValue >= 4: {count}")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

Number of establishments in London with a RatingValue >= 4: 0
None


In [10]:
# Fetch the results using find() and convert to a list
results1 = list(establishments.find(query))

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results1)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [11]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.4769
longitude = 0.0005

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}
sort = sort = [("scores.Hygiene", 1)]
limit = 10

# Print the results
results = list(establishments.find(query).sort(sort).limit(limit))
pprint(results)

[{'AddressLine1': '45 Montpelier Vale',
  'AddressLine2': 'Blackheath',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'BusinessName': 'Ivy Collection',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  'ChangesByServerID': 0,
  'Distance': 4649.307626651096,
  'FHRSID': 1034864,
  'LocalAuthorityBusinessID': 'PI/000295805',
  'LocalAuthorityCode': '523',
  'LocalAuthorityEmailAddress': 'david.edwards@lewisham.gov.uk',
  'LocalAuthorityName': 'Lewisham',
  'LocalAuthorityWebSite': 'http://www.lewisham.gov.uk/myservices/business/food-safety/Pages/default.aspx',
  'NewRatingPending': False,
  'Phone': '',
  'PostCode': '',
  'RatingDate': '2018-01-31T00:00:00',
  'RatingKey': 'fhrs_5_en-gb',
  'RatingValue': 5,
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('679180286278f16cc5ec05c5'),
  'geocode': {'latitude': 51.4676447852657, 'longitude': 0.00813059223847337},
  'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/1034864',
      

In [12]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("\nFirst 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 10

First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  679180286278f16cc5ec05c5  1034864                  0   
1  679180286278f16cc5ec05dd   499238                  0   
2  679180286278f16cc5ec06b7  1047113                  0   
3  679180286278f16cc5ec06b6  1396841                  0   
4  679180286278f16cc5ec05cb   255421                  0   
5  679180286278f16cc5ec082d   940034                  0   
6  679180286278f16cc5ec076b   694874                  0   
7  679180286278f16cc5ec05e0   255505                  0   
8  679180286278f16cc5ec05da  1341447                  0   
9  679180286278f16cc5ec04ce   937547                  0   

  LocalAuthorityBusinessID                         BusinessName  \
0             PI/000295805                       Ivy Collection   
1             PI/000187131                                 Cote   
2             PI/000296126                  Madeleines Creperie   
3            

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [13]:
# Create a pipeline that:
pipeline = [
    # 1. Match establishments with a hygiene score of 0
    {"$match": {"scores.Hygiene": 0}},
    
    # 2. Group by LocalAuthorityName and count the number of establishments
    {"$group": {
        "_id": "$LocalAuthorityName",  # Group by LocalAuthorityName
        "count": {"$sum": 1}  # Count the number of establishments
    }},
    
    # 3. Sort the groups by count in descending order
    {"$sort": {"count": -1}}
]

# Run the aggregation pipeline
results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"Number of Local Authorities with hygiene score of 0: {len(results)}")

# Print the first 10 results
print("\nFirst 10 results:")
for result in results[:10]:
    print(result)


Number of Local Authorities with hygiene score of 0: 55

First 10 results:
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [14]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("\nFirst 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 55

First 10 rows of the DataFrame:
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
